In [22]:
# Importing the needed libraries:
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate as integrate
import scipy.stats as stats
import math
import statistics

# For debug purposes only. To be removed.
import time

In [3]:
# Meixner process parameters. No subordinator.
a = 0.1231
b = -0.5875
d = 3.3588

kappa = 0.5705
eta = 1.5863
lamb = 1.9592
y0 = 1

r = 1.9/100 
q = 1.2/100

S0 = 1124.47

In [4]:
def ch_price(t, K):  
    coth = lambda x : np.cosh(x)/np.sinh(x) 
    
    def phi_cir(u):
        gamma = np.sqrt(complex(kappa**2, -2*lamb**2*u))
        return np.exp(kappa**2*eta*t/(lamb**2))*np.exp(complex(0,2*y0*u)/(kappa + gamma*coth(gamma*t/2)))/((np.cosh(gamma*t/2) + kappa/gamma*np.sinh(gamma*t/2))**(2*kappa*eta/lamb**2))
    
    def psi_X(u):
        return np.log((np.cos(b/2)/np.cosh(complex(a*u,-b)/2))**(2*d))
    
    def phi(u):
        return np.exp(complex(0, u*((r-q)*t+np.log(S0))))*(phi_cir(complex(0,-psi_X(u))))/(phi_cir(complex(0,-psi_X(complex(0,-1)))))**complex(0,u)
    
    integrand1 = lambda u : np.real(np.exp(complex(0,-u)*np.log(K))*phi(complex(u,-1))/complex(0,u*phi(complex(0, -1))))
    integrand2 = lambda u : np.real(np.exp(complex(0,-u)*np.log(K))*phi(u)/complex(0,u))
    
    pi1 = 1/2 + 1/np.pi*(integrate.quad(integrand1, 0, 1000)[0])
    pi2 = 1/2 + 1/np.pi*(integrate.quad(integrand2, 0, 1000)[0])
    
    #return S0*pi1-K*np.exp(-r*t)*pi2
    return S0*np.exp(-q*t)*pi1-K*np.exp(-r*t)*pi2

In [5]:
ch_price(338/365, 1150)

68.37017845752177

In [6]:
K_list = [1025,1100,1125,1150,1175,1200,1225,1250,1275,1300,1325]
prices_list = [146.50,96.20,81.7,68.30,56.6,46.1,36.9,29.3,22.5,17.2,12.8]

In [11]:
cf_predictions = []
for k in K_list:
    cf_predictions.append(ch_price(338/365, k))
the_reveal = list(zip(cf_predictions, prices_list))

In [12]:
the_reveal

[(146.2682038274154, 146.5),
 (96.27854381660791, 96.2),
 (81.71991732622723, 81.7),
 (68.37017845752177, 68.3),
 (56.32144583474735, 56.6),
 (45.65767220010889, 46.1),
 (36.445043138549465, 36.9),
 (28.714744413693666, 29.3),
 (22.431730479365115, 22.5),
 (17.459460290552528, 17.2),
 (13.583692922767995, 12.8)]

In [13]:
arpe = 1/len(the_reveal)*sum([abs(market - model)/market for model, market in the_reveal])

In [14]:
arpe*100

1.180323208549623

In [30]:
def BS_eur_call(T, K):
    sigma = 0.1479
    D1 = (np.log(S0/K) + (r - q +sigma**2/2)*(T))/(sigma*np.sqrt(T))
    D2 = D1 - sigma*np.sqrt(T)
    call = np.exp(-q*T)*S0*stats.norm.cdf(D1) - K*np.exp(-r*T)*stats.norm.cdf(D2)
    return call

In [31]:
BS_predictions = []
for k in K_list:
    BS_predictions.append(BS_eur_call(338/365, k))
BS_comp = list(zip(BS_predictions, prices_list))

In [32]:
BS_comp

[(126.54488928955664, 146.5),
 (79.0555629606738, 96.2),
 (66.28698763237117, 81.7),
 (55.044924877164476, 68.3),
 (45.27231353292541, 56.6),
 (36.88317132044091, 46.1),
 (29.769743081677916, 36.9),
 (23.809957392336457, 29.3),
 (18.874516807101827, 22.5),
 (14.833129707541787, 17.2),
 (11.559587345897597, 12.8)]

In [33]:
BS_arpe = np.mean([abs(market - model)/market for model, market in BS_comp])

In [34]:
BS_arpe*100

17.03159125366254